In [10]:
import pandas as pd
import ast

import torch
import torch.nn.functional as F

from torch_geometric.data import HeteroData
from torch_geometric.nn import GATConv, Linear, to_hetero

import wandb

from sklearn.metrics import r2_score

In [11]:
def convert_edges(edges, mapping_1, mapping_2):
    edges_ = torch.empty((2, len(edges)), dtype=torch.long)
    for i, row in edges.iterrows():
        try:
            edges_[0][i] = mapping_1[row.iloc[0]]
            edges_[1][i] = mapping_2[row.iloc[1]]
        except:
            edges_[0][i] = -1
            edges_[1][i] = -1

    # Remove invalid edges
    edges_ = edges_[:, edges_[0] != -1]

    return edges_


def expand_columns(df, cols):
    num_cols_ = {}
    for col in cols:
        df[col] = df[col].apply(ast.literal_eval)

        num_cols = len(df[col][0])
        num_cols_[col] = num_cols

        cols = pd.DataFrame(
            df[col].tolist(), columns=[f"{col}_{i}" for i in range(num_cols)]
        )

        df = pd.concat([df, cols], axis=1)

        df = df.drop(columns=[col])

    return df, num_cols_


def create_hetero_data(path):
    movies = pd.read_csv(f"{path}/movies.csv")
    actors = pd.read_csv(f"{path}/actors.csv")
    directors = pd.read_csv(f"{path}/directors.csv")
    writers = pd.read_csv(f"{path}/writers.csv")

    movie_actor_ = pd.read_csv(f"{path}/movie_actor.csv")
    movie_director_ = pd.read_csv(f"{path}/movie_director.csv")
    movie_writer_ = pd.read_csv(f"{path}/movie_writer.csv")

    actor_actor_ = pd.read_csv(f"{path}/actor_actor.csv")
    actor_director_ = pd.read_csv(f"{path}/actor_director.csv")
    actor_writer_ = pd.read_csv(f"{path}/actor_writer.csv")

    director_director_ = pd.read_csv(f"{path}/director_director.csv")
    director_writer_ = pd.read_csv(f"{path}/director_writer.csv")

    writer_writer_ = pd.read_csv(f"{path}/writer_writer.csv")

    movies, num_cols = expand_columns(movies, ["genres"])

    movies["release_date"] = pd.to_datetime(movies["release_date"])
    first_date = movies["release_date"].min()
    movies["release_date"] = (movies["release_date"] - first_date).dt.days

    movies_id = movies["id"].tolist()
    actors_id = actors["id"].tolist()
    directors_id = directors["id"].tolist()
    writers_id = writers["id"].tolist()

    movies_map = dict(zip(movies_id, range(len(movies_id))))
    actors_map = dict(zip(actors_id, range(len(actors_id))))
    directors_map = dict(zip(directors_id, range(len(directors_id))))
    writers_map = dict(zip(writers_id, range(len(writers_id))))

    data = HeteroData()

    data["movie"].ids = torch.tensor(movies_id, dtype=torch.long)
    data["movie"].x = torch.tensor(
        movies[
            [
                "release_date",
                "budget",
                "revenue",
                "vote_average",
                "vote_count",
                "popularity",
            ]
            + [f"genres_{i}" for i in range(num_cols["genres"])]
        ].values.tolist(),
        dtype=torch.float,
    )

    # normalize
    data["movie"].x[:, :6] = F.normalize(data["movie"].x[:, :6], dim=0)
    data["movie"].x[:, 6:] = F.normalize(data["movie"].x[:, 6:], dim=1)

    data["actor"].ids = torch.tensor(actors_id, dtype=torch.long)
    data["actor"].x = torch.ones((len(actors_id), 1), dtype=torch.float)
    data["actor"].y = torch.tensor(
        actors[["popularity"]].values.tolist(), dtype=torch.float
    )

    data["actor"].y = torch.log(data["actor"].y)

    data["director"].ids = torch.tensor(directors_id, dtype=torch.long)
    data["director"].x = torch.tensor(
        directors[["popularity"]].values.tolist(), dtype=torch.float
    )

    data["writer"].ids = torch.tensor(writers_id, dtype=torch.long)
    data["writer"].x = torch.tensor(
        writers[["popularity"]].values.tolist(), dtype=torch.float
    )

    data["movie", "played_by", "actor"].edge_index = convert_edges(
        movie_actor_, movies_map, actors_map
    )

    data["movie", "directed_by", "director"].edge_index = convert_edges(
        movie_director_, movies_map, directors_map
    )
    data["movie", "written_by", "writer"].edge_index = convert_edges(
        movie_writer_, movies_map, writers_map
    )

    data["actor", "same_page", "actor"].edge_index = convert_edges(
        actor_actor_, actors_map, actors_map
    )
    data["actor", "same_page", "director"].edge_index = convert_edges(
        actor_director_, actors_map, directors_map
    )
    data["actor", "same_page", "writer"].edge_index = convert_edges(
        actor_writer_, actors_map, writers_map
    )

    data["director", "same_page", "director"].edge_index = convert_edges(
        director_director_, directors_map, directors_map
    )
    data["director", "same_page", "writer"].edge_index = convert_edges(
        director_writer_, directors_map, writers_map
    )

    data["writer", "same_page", "writer"].edge_index = convert_edges(
        writer_writer_, writers_map, writers_map
    )

    return data

In [12]:
data = create_hetero_data("dataset")

In [13]:
print(data)

HeteroData(
  movie={
    ids=[13337],
    x=[13337, 25],
  },
  actor={
    ids=[54506],
    x=[54506, 1],
    y=[54506, 1],
  },
  director={
    ids=[9291],
    x=[9291, 1],
  },
  writer={
    ids=[11412],
    x=[11412, 1],
  },
  (movie, played_by, actor)={ edge_index=[2, 96751] },
  (movie, directed_by, director)={ edge_index=[2, 12357] },
  (movie, written_by, writer)={ edge_index=[2, 14432] },
  (actor, same_page, actor)={ edge_index=[2, 119604] },
  (actor, same_page, director)={ edge_index=[2, 33168] },
  (actor, same_page, writer)={ edge_index=[2, 30598] },
  (director, same_page, director)={ edge_index=[2, 7394] },
  (director, same_page, writer)={ edge_index=[2, 8201] },
  (writer, same_page, writer)={ edge_index=[2, 7395] }
)


In [14]:
import torch_geometric.transforms as T

data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.conv2 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        return x


data.to(device)

model = GNN(hidden_channels=128, out_channels=1)
model = to_hetero(model, data.metadata(), aggr="mean").to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

random_indices = torch.randperm(data["actor"].num_nodes)
train_indices = random_indices[: int(data["actor"].num_nodes * 0.8)]
test_indices = random_indices[int(data["actor"].num_nodes * 0.8) :]

wandb.init(project="movie-actor")


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    loss = F.mse_loss(out["actor"][train_indices], data["actor"].y[train_indices])
    loss.backward()
    optimizer.step()
    return float(loss)


for epoch in range(1, 1001):
    loss = train()
    print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}")
    wandb.log({"training-loss": loss})

    if epoch % 10 == 0:
        model.eval()
        out = model(data.x_dict, data.edge_index_dict)

        y_pred = out["actor"][train_indices].detach().cpu().numpy()
        y_true = data["actor"].y[train_indices].detach().cpu().numpy()
        score = r2_score(y_true, y_pred)
        wandb.log({"train-score": score})

        y_pred = out["actor"][test_indices].detach().cpu().numpy()
        y_true = data["actor"].y[test_indices].detach().cpu().numpy()
        score = r2_score(y_true, y_pred)
        wandb.log({"test-score": score})

        test_loss = F.mse_loss(
            out["actor"][test_indices], data["actor"].y[test_indices]
        )
        wandb.log({"test-loss": test_loss})

        print(f"Epoch: {epoch:03d}, loss: {test_loss:.4f}, score: {score:.4f}")

wandb.finish()

Epoch: 001, Loss: 214.8935
Epoch: 002, Loss: 212.0808
Epoch: 003, Loss: 209.3150
Epoch: 004, Loss: 206.3878
Epoch: 005, Loss: 203.0734
Epoch: 006, Loss: 199.3001
Epoch: 007, Loss: 195.0981
Epoch: 008, Loss: 190.4244
Epoch: 009, Loss: 185.1948
Epoch: 010, Loss: 179.3391
Epoch: 010, loss: 181.8727, score: -0.2460
Epoch: 011, Loss: 172.7906
Epoch: 012, Loss: 165.4415
Epoch: 013, Loss: 157.2209
Epoch: 014, Loss: 148.3226
Epoch: 015, Loss: 138.3110
Epoch: 016, Loss: 128.3773
Epoch: 017, Loss: 118.9351
Epoch: 018, Loss: 111.4832
Epoch: 019, Loss: 107.4496
Epoch: 020, Loss: 107.0978
Epoch: 020, loss: 112.4844, score: 0.2294
Epoch: 021, Loss: 108.5304
Epoch: 022, Loss: 110.7630
Epoch: 023, Loss: 112.3195
Epoch: 024, Loss: 112.1562
Epoch: 025, Loss: 110.3277
Epoch: 026, Loss: 107.4334
Epoch: 027, Loss: 104.2009
Epoch: 028, Loss: 101.2034
Epoch: 029, Loss: 98.7648
Epoch: 030, Loss: 96.9979
Epoch: 030, loss: 100.3832, score: 0.3123
Epoch: 031, Loss: 95.8446
Epoch: 032, Loss: 95.1447
Epoch: 033, L

test-loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test-score,▁▇▇▇████████████████████████████████████
train-score,▁▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████████████
training-loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test-loss,85.90762
test-score,0.41146
train-score,0.50655
training-loss,67.467
